# Reproducing Bradley Terry Model Figure 1

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import pyarrow as pa
import joypy
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
from matplotlib import cm

sys.path.append("/Users/ryanschenck/Dropbox/Projects/FAP_Project/mutation_timing")
from BTmodel import run

font = {'size'   : 8}
plt.rc('font', **font)
plt.rcParams['pdf.fonttype'] = 42

## Load resources

List of known drivers used as potential genes in Bradley Terry model.

In [ ]:
def get_cancer_drivers(driverfile="data/resource/PanCanDrivers_Cell2018.csv"):
    x = pd.read_csv(driverfile,skiprows=3)
    listofgenes = x["Gene"]
    return (list(set(listofgenes)))

drivers = get_cancer_drivers()

## Read Data

In [ ]:
htan = "data/combined_noshared_FILTERED_muts_WGS.csv"
df = pd.read_csv(htan, low_memory=False)
df['WXS'] = 'WGS'

htanwes = "data/combined_noshared_FILTERED_muts_WES.csv"
dfwes = pd.read_csv(htanwes, low_memory=False)
dfwes['WXS'] = 'WES'

df = pd.concat([df, dfwes]).reset_index(drop=True)

### Prepare data

In [ ]:
# Ensure CLONAL/SUBCLONAL are strings and reduce clonality calls to SUBCLONAL/CLONAL
df['binary_clonal'] = df['purity_clonal'].apply(lambda x: 0 if 'SUBCLONAL' in x else 1)
df['clonality'] = df['purity_clonal'].apply(lambda x: 'SUBCLONAL' if 'SUBCLONAL' in x else 'CLONAL')

# Only use these columns
df = df[['Patient', 'Mut_ID', 'WXS', 'Tumor_Sample_Barcode', 'Hugo_Symbol', 'Driver', 'Stage', 'clonality', 'binary_clonal', 'tcn.sequenza', 'ln.sequenza']].reset_index(drop=True)

# Take only WGS mutations where WES and WGS are both available
dups = df[df.duplicated(subset=['Patient', 'Mut_ID', 'Tumor_Sample_Barcode', 'Hugo_Symbol', 'Driver', 'Stage'], keep=False)].reset_index(drop=True)
nondups = df[~df.duplicated(subset=['Patient', 'Mut_ID', 'Tumor_Sample_Barcode', 'Hugo_Symbol', 'Driver', 'Stage'], keep=False)].reset_index(drop=True)

# Keep only the WGS record for duplicates
dups = dups[dups['WXS']=='WGS'].reset_index(drop=True)

# Combine the two
df = pd.concat([nondups, dups]).reset_index(drop=True)



### Prepare Li et al. data

In [ ]:
pku = '/Users/ryanschenck/Dropbox/Projects/FAP_Project/data/HTAN_FAP_MAFs/snpconsensus/PKU_samples.snpconsensus.filtered.mpileups_filtered.ccfs.ccfs_noprobs.maf'
dfpku = pd.read_csv(pku, sep='\t', low_memory=False)[['Patient', 'Mut_ID', 'WXS', 'Tumor_Sample_Barcode', 'Hugo_Symbol', 'Driver',  'Stage', 'ccf_expected_copies', 'clonality', 'tcn.sequenza', 'ln.sequenza']]

dfpku['binary_clonal'] = dfpku['clonality'].apply(lambda x: 0 if 'SUBCLONAL' in str(x) else 1)
dfpku.rename(columns={'ccf_expected_copies':'purity_ccf', 'binary_clonal':'binary_clonal'}, inplace=True)
dfpku = dfpku[['Patient', 'Mut_ID', 'WXS', 'Tumor_Sample_Barcode', 'Hugo_Symbol', 'Driver', 'Stage', 'clonality', 'binary_clonal', 'tcn.sequenza', 'ln.sequenza']].reset_index(drop=True)

# Break Tumor Sample Barcode into Patient and Sample and Region
dfpku['Sample'] = dfpku['Tumor_Sample_Barcode'].str.split('_').str[1]
dfpku['Region'] = dfpku['Tumor_Sample_Barcode'].str.split('_').str[2]

# Get duplicates for dfpku based on Sample and Region
dups = dfpku[dfpku.duplicated(subset=['Patient', 'Mut_ID', 'Sample', 'Hugo_Symbol', 'Driver', 'Stage'], keep=False)].reset_index(drop=True)
nondups = dfpku[~dfpku.duplicated(subset=['Patient', 'Mut_ID', 'Sample', 'Hugo_Symbol', 'Driver', 'Stage'], keep=False)].reset_index(drop=True)

# Keep the 'CLONAL' record for duplicates as maximum observed, or subclonal if no clonal present
keepers = []
for s, sdf in dups.groupby(['Patient','Sample','Mut_ID']):
    # Check if there's a clonal record clonality
    if 'CLONAL' in sdf['clonality'].values:
        clonal = sdf[sdf['clonality']=='CLONAL']
        if len(clonal) > 1:
            # Take only the first row index
            idx = clonal.index[0]
            keepers.append(idx)
        else:
            keepers.append(clonal.index[0])
    else:
        # Take first row
        keepers.append(sdf.index[0])

keepers = dups.loc[keepers]

# Combine cleaned dataset
dfpku = pd.concat([nondups, keepers]).reset_index(drop=True)

# Check to make sure no duplicates now
dups = dfpku[dfpku.duplicated(subset=['Patient', 'Mut_ID', 'Sample', 'Hugo_Symbol', 'Driver', 'Stage'], keep=False)].reset_index(drop=True)
assert dups.shape[0] == 0, 'Duplicates still present, check code'

## Subset Drivers

In [ ]:
# Driver status changed to boolean (previous iterations of SNVs had it as a string/boolean/binary)

# HTAN DATA
htan_drivers = df[(df['Hugo_Symbol'].isin(drivers)) & (df['Driver']==True)].reset_index(drop=True)
htan_drivers['Driver'] = htan_drivers['Driver'].apply(lambda x: True if x else False)

# PKU DATA
pku_drivers = dfpku[(dfpku['Hugo_Symbol'].isin(drivers)) & (dfpku['Driver']==True)].reset_index(drop=True)
pku_drivers['Driver'] = pku_drivers['Driver'].apply(lambda x: True if x else False)

In [ ]:
# Add cohort column
htan_drivers['Cohort'] = 'HTAN'
pku_drivers['Cohort'] = 'PKU'

# Combine
driverDf = pd.concat([htan_drivers, pku_drivers]).reset_index(drop=True)

## Remove Mucosa samples

In [ ]:
driverDf = driverDf[~driverDf['Stage'].str.contains('Mucosa')].reset_index(drop=True)

## Filter for the number of driver mutations per gene

In [ ]:
total_counts = driverDf['Hugo_Symbol'].value_counts()
# Sort by counts
total_counts = total_counts.sort_values(ascending=False).reset_index(drop=False)
total_counts.columns = ['Hugo_Symbol','Count']

# Counts by cohort
cohort_counts = driverDf.groupby(['Hugo_Symbol','Cohort']).size().reset_index(drop=False).rename(columns={0:'Count'})
# Sort by Hugo_Symbol and Cohort

# Get counts for each stage
stage_counts = driverDf.groupby(['Hugo_Symbol','Stage']).size().reset_index(drop=False).rename(columns={0:'Count'})
# Sort by Hugo_Symbol and Cohort
stage_counts = stage_counts.sort_values(by=['Hugo_Symbol','Stage']).reset_index(drop=True)

# Get counts by Stage and Cohort
stage_cohort_counts = driverDf.groupby(['Hugo_Symbol','Stage','Cohort']).size().reset_index(drop=False).rename(columns={0:'Count'})
# Sort by Hugo_Symbol and Cohort
stage_cohort_counts = stage_cohort_counts.sort_values(by=['Hugo_Symbol','Stage','Cohort']).reset_index(drop=True)

### Plot the number of driver mutations per gene

In [ ]:
# Plot each as stacked bar for each cohort
fig, ax = plt.subplots(3,1,figsize=(16,10))

def plot_counts(data, ax, group=None, title="", threshold=0):
    # Ensure data sorted by count
    data = data.sort_values(by='Count', ascending=False).reset_index(drop=True)

    if group is None:
        # Add group based on threshold
        data['Driver Count'] = data['Count'].apply(lambda x: f'≥{threshold}' if x > threshold else f'<{threshold}')
        sns.barplot(x='Hugo_Symbol', y='Count', data=data, hue='Driver Count', ax=ax, palette='Set2')

        # Add count to top of each bar
        for p in ax.patches:
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=6)
    else:
        # Stacked barchart by group
        sns.barplot(x='Hugo_Symbol', y='Count', data=data, hue=group, ax=ax, palette='Set2')
        

    # Rotate x labels
    for item in ax.get_xticklabels():
        item.set_rotation(90)
    ax.set_title(title)

    # Log scale
    ax.set_yscale('log')

    # Ax
    ax.set_xlabel('')

plot_counts(total_counts, ax[0], group=None, title='All FAP cohorts and stages', threshold=1)

plot_counts(cohort_counts, ax[1], group='Cohort', title='All FAP cohorts by cohort', threshold=1)

plot_counts(stage_counts, ax[2], group='Stage', title='All FAP cohorts by stage', threshold=1)

sns.despine()
plt.tight_layout()
plt.show()

# Bradley Terry Model Functions

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from tqdm import tqdm
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns

def bradleyterry(df, wd="analysis/bradley_terry/preferential_ordering/wd"):
    df.to_csv("%s/tmp.csv" % (wd), index=False)
    cmd = "Rscript analysis/bradley_terry/preferential_ordering/bradleyterry.R %s/tmp.csv %s" % (wd, wd)

    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    process.wait()

    if process.returncode==1:
        print(process.stdout.read().decode("utf-8"))
        print(process.stderr.read().decode("utf-8"))
        print(df.shape)
        raise Exception("Error in R script")
    else:
        os.remove("%s/tmp.csv" % (wd))
        df = pd.read_csv("%s/tmp_out.csv" % (wd))
        os.remove("%s/tmp_out.csv" % (wd))
        return(df)

def get_cancer_drivers(driverfile="data/resource/PanCanDrivers_Cell2018.csv"):
    x = pd.read_csv(driverfile,skiprows=3)
    listofgenes = x["Gene"]
    return (list(set(listofgenes)))

def frequencyOfWinners(df, by='Tumor_Sample_Barcode', clonality_col='clonality'):
    genes = df['Hugo_Symbol'].unique().tolist()

    # calculate frequency of gene 1 occuring before gene 2
    ret = []
    pairs = []
    samples_with_pair = []
    n_sample_in_pair = []

    n=1
    for i in range(len(genes)):
      # print(i)
      # select mutations of interest

      gene_1 = genes[i]

      for j in range(i+1, len(genes)):
        # print(j)
        # j <- 2
        gene_2 = genes[j]

        if gene_1 != gene_2:
            # if gene_1!='toss' and gene_2!='toss':
            # record two pairs
            pairs.append((gene_1, gene_2))

            # select patients with both mutations
            samplesWithMuts = df[(df['Hugo_Symbol'].isin([gene_1, gene_2]))]
            groups = samplesWithMuts.groupby(by)

            samplesRet = []
            n_1_before_2 = 0
            n_2_before_1 = 0
            for g, gdf in groups:
                clonality_gene1 = gdf[(gdf['Hugo_Symbol'] == gene_1) & (
                        gdf[clonality_col] == 'CLONAL')].shape[0]
                clonality_gene2 = gdf[(gdf['Hugo_Symbol'] == gene_2) & (
                        gdf[clonality_col] == 'CLONAL')].shape[0]
                if gdf['Hugo_Symbol'].unique().shape[0]>=2:
                    samplesRet.append(gdf)

                    if clonality_gene1 > clonality_gene2:
                        n_1_before_2+=1
                    elif clonality_gene1 < clonality_gene2:
                        n_2_before_1+=1
                    else:
                        pass
                else:
                    # This is where the mutation occurs as clonal without the other...this should be a win.
                    if clonality_gene1>0:
                        n_1_before_2 += 1
                    elif clonality_gene1>0:
                        n_2_before_1 += 1
                    else:
                        pass

            ret.append(pd.DataFrame({ 'gene_1':[gene_1], 'gene_2':[gene_2], 'n_1_before_2':[n_1_before_2], 'n_2_before_1':[n_2_before_1] }))
    if len(ret)>0:
        ret = pd.concat(ret)
        # Remove cases with zero wins in the gene1 vs gene2 pair
        counts = ret[ret[['n_1_before_2', 'n_2_before_1']].sum(axis=1) > 0].reset_index(drop=True)
        # if counts[counts['gene_1']=='DMD'].shape[0] > 0:
        #     print('DMD Found')
        return (counts)
    else:
        return (pd.DataFrame())

def loo_loop(gdf, ret, by, group, wd="analysis/bradley_terry/preferential_ordering/wd", clonality_col='clonality'):
    '''
    Leave one out loop for the Bradley Terry model

    :param gdf: dataframe of mutation winners
    :param ret: list to hold results
    :param by: factor to group by (e.g. 'Stage')
    :param group: group name (e.g. 'Dysplasia')
    :return: ret, list of results now filled
    '''
    samples = gdf['Tumor_Sample_Barcode'].unique().tolist()
    for i in tqdm(range(len(samples)), desc="Running leave one out loop for %s" % (group)):
        gdf_all_but_1 = gdf[gdf['Tumor_Sample_Barcode'] != samples[i]]
        # Get frequency of winners
        frequencies = frequencyOfWinners(gdf_all_but_1, clonality_col=clonality_col)
        if frequencies.shape[0] > 1:
            bt_out = bradleyterry(frequencies, wd=wd)
            bt_out[by] = group
            bt_out['loo_sample'] = samples[i]
            ret.append(bt_out)
        # else:
            # print("No winners for %s" % (group))
    return(ret)

def bootstrap_loop(gdf, ret, by, group, n=100, wd="analysis/bradley_terry/preferential_ordering/wd", clonality_col='clonality'):
    samples = gdf['Tumor_Sample_Barcode'].unique().tolist()
    for i in tqdm(range(n)):
        samples_to_run = np.random.choice(samples, int(np.floor(len(samples)*.75)), replace=False)
        gdf_boot = [gdf[gdf['Tumor_Sample_Barcode'] == samples_to_run[j]].reset_index(drop=True) for j in range(len(samples_to_run))]
        boot_samples = []
        for k, sam in enumerate(gdf_boot):
            sam['boot_sam_num'] = k
            boot_samples.append(sam)
        gdf_boot = pd.concat(boot_samples, axis=0).reset_index(drop=True)
        # Get frequency of winners
        frequencies = frequencyOfWinners(gdf_boot, by='boot_sam_num', clonality_col=clonality_col)
        if frequencies.shape[0] > 1:
            bt_out = bradleyterry(frequencies, wd=wd)
            bt_out[by] = group
            bt_out['bootstrap_run'] = i
            ret.append(bt_out)
        else:
            print("No winners for %s" % (group))
    return (ret)

def run(df, allSamples=True, by="Stage", bootstrap=False, wd="analysis/bradley_terry/preferential_ordering/wd", loo_func=False, clonality_col='clonality'):
    '''
    Runs the bradley terry model

    :param df: dataframe of mutations with clonality
    :param allSamples: boolean, if True, run on all samples together, if False, run on samples with at least 2 mutations
    :param by: factor to group by (e.g. 'Stage'), must be specified if allSamples=False
    :param bootstrap: boolean, if True, run leave one out loop
    :return: Bradley Terry results
    '''
    if allSamples and bootstrap==False:
        # Run on all samples together
        frequencies = frequencyOfWinners(df, clonality_col=clonality_col)
        bt_out = bradleyterry(frequencies, wd=wd)
        bt_out[by] = 'All Samples'
        return (bt_out)
    elif allSamples and bootstrap:
        # run bootstrap loop on all samples
        ret = []
        if loo_func:
            ret = loo_loop(df, ret, by, 'All Samples', wd=wd, clonality_col=clonality_col)
        else:
            ret = bootstrap_loop(df, ret, by, 'All Samples', wd=wd, clonality_col=clonality_col)
        if len(ret) > 0:
            bt_out = pd.concat(ret, axis=0).reset_index(drop=True)
            bt_out[by] = 'All Samples'
            return (bt_out)
        else:
            return (np.nan)
    elif allSamples==False and bootstrap==False:
        # Run by stage
        ret = []
        for group, gdf in df.groupby(by):
            print("Running group: %s\n" % (group))
            if gdf.shape[0]>0:
                # Get frequency of winners
                frequencies = frequencyOfWinners(gdf, clonality_col=clonality_col)
                if frequencies.shape[0]>0:
                    bt_out = bradleyterry(frequencies, wd=wd)
                    bt_out[by] = group
                    ret.append(bt_out)
                else:
                    print("No winners for %s" % (group))
            else:
                print("No samples in group %s" % (group))
        if len(ret)>0:
            ret = pd.concat(ret, axis=0).reset_index(drop=True)
            return ( ret )
        else:
            return (np.nan)
    elif allSamples==False and bootstrap:
        ret = []
        for group, gdf in df.groupby(by):
            print("Running group: %s\n" % (group))
            if gdf.shape[0] > 0:
                # Run the per sample leave one out loop
                # ret = bootstrap_loop(gdf, ret, by, group, wd=wd)
                if loo_func:
                    ret = loo_loop(gdf, ret, by, group, wd=wd, clonality_col=clonality_col)
                else:
                    ret = bootstrap_loop(gdf, ret, by, group, wd=wd, clonality_col=clonality_col)
            else:
                print("No samples in group %s" % (group))
        if len(ret) > 0:
            ret = pd.concat(ret, axis=0).reset_index(drop=True)
            return (ret)
        else:
            return (np.nan)
    else:
        raise Exception("Invalid combination of parameters")

def plotResults(btdf):
    n_stages = btdf['Stage'].unique().shape[0]

    fig, axs = plt.subplots(n_stages, 1, figsize=(10, 10))
    i = 0
    for g, groupdf in btdf.groupby('Stage'):
        groupdf = groupdf[groupdf['Pr(>|z|)'] <= 0.05]
        groupdf = groupdf.sort_values('Estimate', ascending=True)

        if groupdf.shape[0] > 0:
            ax = axs[i]
            sns.boxplot(x='gene', y='rankOrder', data=groupdf, ax=ax)
            ax.set_title(g)
            ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
            ax.set_xlabel('')
        i += 1
    plt.tight_layout()
    plt.show()

# def main():
#     # Get system arguments for maf file and working directory
#     maf_file = sys.argv[1] # File with mutations, this should be a parquet file
#     wd = sys.argv[2] # Working directory
#     outname = sys.argv[3] # Output name

#     # maf_file = "/Users/ryanschenck/Dropbox/Projects/FAP_Project/data/vcfs/FAP_samples.correctedCCF.parquet"
#     # wd = "/Users/ryanschenck/Dropbox/Projects/FAP_Project/mutation_timing"
#     # outname = "loo_bootstrap"

#     # Load the maf file
#     mutDf = pq.read_pandas(maf_file).to_pandas()
#     # mutDf = mutDf[mutDf['Variant_Classification'].isin(['Missense_Mutation', 'Nonsense_Mutation', 'Translation_Start_Site'])]
#     mutDf = mutDf[mutDf['Driver']==True]

#     # Load known drivers
#     drivers = get_cancer_drivers()
#     # drivers = drivers[0:20] # subset for dev testing

#     driverDf = mutDf[mutDf['Hugo_Symbol'].isin(drivers)].reset_index(drop=True)

#     # Run across all samples combined
#     loopFunc = False
#     print("Running BT across all samples")
#     if outname == "loo":
#         loopFunc = True
#     btretAllFAPSamples = run(driverDf, allSamples=True, bootstrap=True, by="Stage2", wd=wd, loo_func=loopFunc)

#     # Run across samples by stage
#     print("Running BT across each stage")
#     driverDf['Stage2'] = driverDf['Stage'].apply(lambda x: "Dysplasia + AdCa" if x in ['Dysplasia', 'AdCa'] else x)
#     if outname == "loo":
#         loopFunc = True
#     btret = run(driverDf, allSamples=False, by="Stage2", bootstrap=True, wd=wd, loo_func=loopFunc)

#     # Combine Runs
#     btret = pd.concat([btretAllFAPSamples, btret], axis=0).reset_index(drop=True)

#     # Output results for visualizations
#     outputfile = os.path.basename(maf_file).replace('.parquet', '.bradleyterry.') + outname + '.csv'
#     btret.to_csv("%s/%s" % (wd, outputfile), index=False)

#     print(btret)

# Execute Bradley Terry Model without thresholding

In [ ]:
outname = "loo_bootstrap"
wd = "analysis/bradley_terry/preferential_ordering/wd/"

# Run across all samples combined
loopFunc = False

# Run across samples by stage
print("Running BT across each stage")
driverDf['Stage2'] = driverDf['Stage'].apply(lambda x: "Dysplasia" if x in ['Dysplasia'] else x)
if outname == "loo":
    loopFunc = True
btret = run(driverDf, allSamples=False, by="Stage2", bootstrap=True, wd=wd, loo_func=loopFunc, clonality_col='clonality', threshold=1)

## Bradley Terry Model with thresholding

In [ ]:
thresh = 2

# Subset where count hugo_symbol is present more than once in total_counts and stage_counts
total_counts_subset = total_counts[total_counts['Count']>thresh]['Hugo_Symbol'].unique().tolist()

benign_thresh_genes = stage_counts[(stage_counts['Count']>thresh) & (stage_counts['Stage'].isin(['Benign']))]['Hugo_Symbol'].unique().tolist()

dysplasia_thresh_genes = stage_counts[(stage_counts['Count']>thresh) & (stage_counts['Stage'].isin(['Dysplasia', 'AdCa']))]['Hugo_Symbol'].unique().tolist()

# Subest by stage and count
benign = driverDf[(driverDf['Hugo_Symbol'].isin(benign_thresh_genes)) & (driverDf['Stage'].isin(['Benign']))].reset_index(drop=True)

# Subset by stage and count
dysplasia = driverDf[(driverDf['Hugo_Symbol'].isin(dysplasia_thresh_genes)) & (driverDf['Stage'].isin(['Dysplasia', 'AdCa']))].reset_index(drop=True)

# Combine
driverDfSubset = pd.concat([benign, dysplasia]).reset_index(drop=True)

In [ ]:
outname = "loo_bootstrap.thresh_2"
wd = "analysis/bradley_terry/preferential_ordering/wd"

# # Run across all samples combined
loopFunc = False

# Run across samples by stage
print("Running BT across each stage")
driverDfSubset['Stage2'] = driverDfSubset['Stage'].apply(lambda x: "Dysplasia" if x in ['Dysplasia','AdCa'] else x)
# driverDfBCI['Stage2'] = driverDfBCI['Stage']
if outname == "loo":
    loopFunc = True
btret = run(driverDfSubset, allSamples=False, by="Stage2", bootstrap=True, wd=wd, loo_func=loopFunc, clonality_col='clonality')

In [ ]:
# Save results
# Output results for visualizations
outputfile = f'{outname}.bradleyterry.csv'
btret.to_csv("%s" % (outputfile), index=False)

# Filter and Plot Results

In [ ]:
bootstrap_method = 'bootstrap_run' # loo_sample or bootstrap_run
statistic = 'Pr(>|z|)' # fdr or Pr(>|z|)

combined_datasets = []
dataset_names = ['FAP']
for dataidx, dataset in enumerate([outputfile]):
    print(dataset)
    if bootstrap_method == 'bootstrap_run':
        btret = pd.read_csv(dataset, low_memory=False)
        btret['Stage2'] = btret['Stage2'].apply(lambda x: 'All Samples' if pd.isna(x) else x)
        # print(btret)
        # btret.head()
    else:
        btret = pd.read_csv(dataset, low_memory=False)
        
#     # Take genes who had a adjusted p-value is <= 0.01
#     grouped = []
#     for stage, gdf in btret.groupby('Stage2'):
#         print(stage)
#         print(gdf[gdf['gene']=='KRAS'][statistic].min())

#         genesthatpass = gdf[gdf[statistic]<=0.05]['gene'].unique().tolist()
#         gdf = gdf[gdf['gene'].isin(genesthatpass)]
#         grouped.append(gdf)
#     passing_df = pd.concat(grouped, axis=0).reset_index(drop=True)
    
    passing_df = btret[btret['gene'].isin( btret[btret[statistic]<=0.05]['gene'].unique().tolist() )]
    
    # This reduces the reference category (refcat) in the bT model
    passing_df = passing_df.groupby(["gene", bootstrap_method, "Stage2"])['Estimate'].median().reset_index(drop=False)
    
    # Assign percentile rank
    ret = []
    for g, gdf in passing_df.groupby(['Stage2', bootstrap_method]):
        # print(gdf['gene'].shape, gdf['gene'].unique().shape)
        gdf['Rank_Order'] = gdf['Estimate'].rank(ascending=False, pct=True)
        ret.append(gdf)
    ranked_df = pd.concat(ret, axis=0).reset_index(drop=True)
    print(ranked_df)
    combined_datasets.append(ranked_df)
    
    if dataset == '':
        dataset = 'HTAN'
    ranked_df['dataset'] = dataset.replace('/','')

    fig, axs = plt.subplots(ranked_df['Stage2'].unique().shape[0], 1, figsize=(16,10))
    
    i = 0
    for g, gdf in ranked_df.groupby("Stage2"):
        print(gdf)
        mean_rank = gdf.groupby("gene")["Rank_Order"].mean().reset_index(drop=False)
        ordered_genes = mean_rank.sort_values("Rank_Order")['gene'].tolist()
        plot_order = pd.CategoricalDtype(ordered_genes, ordered=True)
        gdf['gene'] = gdf['gene'].astype(plot_order)
        # gdf = gdf.sort_values("Rank_Order")
        
        ybreaks = [0.3, 0.6,] # [0, 0.25, 0.5, .75, 1.0]
        # for pos in ybreaks:
        #     axs[i].axhline(pos, zorder=0, linestyle='--', color="black", alpha=0.2)

        plot = sns.boxplot(x='gene', y='Rank_Order', data=gdf, ax = axs[i], fliersize=0)
        # axs[i].scatter(gdf['gene'], gdf['Rank_Order'])
        axs[i].tick_params(labelrotation=90, axis='x')
        axs[i].set_title(g)
        axs[i].set_ylabel("Rank Order Percent")
        axs[i].set_yticks([0, 0.3, 0.6, 1.0])
        i+=1

    sns.despine()
    plt.tight_layout()
    plt.savefig(f"boxplots_{dataset_names[dataidx]}_{outname}.pdf")
    plt.show()

ranked_df = pd.concat(combined_datasets, axis=0).reset_index(drop=True)